# Practice: question answering with retrieval

In this homework you will build a retrieval-based question answering system, one component at a time.

_Okay, realistically, there's like, two components, but technically it's one component at a time._


![img](https://www.cs.upc.edu/~mlatifi/index_files/qa-logo.jpg)



There are two parts to this type of systems: a retriever and a generator.
- the retriever subsystem searches for similar texts from a given databse, e.g. wikipedia
- the generator uses the texts found by the retriever to generate an answer in natural language

_this seminar is based on original notebook by [Oleg Vasilev](https://github.com/Omrigan/)_



### Retriever training: loss function (2 points)
We want our vectorizers to put correct answers closer to question vectors and incorrect answers farther away from them. One way to express this is to use is Pairwise Hinge Loss _(aka Triplet Loss)_.

$$ L = \frac 1N \underset {q, a^+, a^-} \sum max(0, \space \delta - sim[V_q(q), V_a(a^+)] + sim[V_q(q), V_a(a^-)] )$$

, where
* sim[a, b] is some similarity function: dot product, cosine or negative distance
* δ - loss hyperparameter, e.g. δ=1.0. If sim[a, b] is linear in b, all δ > 0 are equivalent.


This reads as __Correct answers must be closer than the wrong ones by at least δ.__

![img](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/margin.png)
<center>_image: question vector is green, correct answers are blue, incorrect answers are red_</center>


Note: in effect, we train a Deep Semantic Similarity Model [DSSM](https://www.microsoft.com/en-us/research/project/dssm/).

### Optional tasks for bonus points

There are many ways to improve our question answering model. Here's a bunch of things you can do to increase your understanding and get bonus points.

### 1.  Hard Negatives (2+ pts)

Not all wrong answers are equally wrong. As the training progresses, _most negative examples $a^-$ will be to easy._ So easy in fact, that loss function and gradients on such negatives is exactly __0.0__. To improve training efficiency, one can __mine hard negative samples__.

Given a list of answers,
* __Hard negative__ is the wrong answer with highest similarity with question,

$$a^-_{hard} = \underset {a^-} {argmax} \space sim[V_q(q), V_a(a^-)]$$

* __Semi-hard negative__ is the one with highest similarity _among wrong answers that are farther than positive one. This option is more useful if some wrong answers may actually be mislabelled correct answers.

* One can also __sample__ negatives proportionally to $$P(a^-_i) \sim e ^ {sim[V_q(q), V_a(a^-_i)]}$$


The task is to implement at least __hard negative__ sampling and apply it for model training.


### 2. Better prompting (2+ pts)

In the previous example, we manually engineer a prompt for an LLM to solve produce an answer. However, by this point you know multiple ways to make LLM do your bidding. In this assignment, you should try at least some of them:
- try few-shot learning with several handcrafted examples (or hand-picked model inputs)
- compare several instruct and/or non-instruct models; for non-instruct models
  - please not that you should not use apply_chat_template for non-instruct models
- provide some means of quality evaluation to compare your approach against the default one

At the minimum, several (10-20) side-by-side examples would do the trick. However, we'd appreciate creative means of evaluation here (crowdsourcing, asking another LM, anything exotic as long as you can explain it).

### 3. Search engine (3+ pts)

Our basic model only selects answers from 2-5 available sentences in paragraph. You can extend it to search over __the whole dataset__. All sentences in all other paragraphs are viable answers.

The goal is to train such a model and use it to __quickly find top-10 answers from the whole set__.

* You can ask such model a question of your own making - to see which answers it can find among the entire training dataset or even the entire wikipedia.
* Searching for top-K neighbors is easier if you use specialized methods: [KD-Tree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html) or [HNSW](https://github.com/nmslib/hnswlib).
* This task is much easier to train if you use hard or semi-hard negatives. You can even find hard negatives for one question from correct answers to other questions in batch - do so in-graph for maximum efficiency. See [1.] for more details.


# Doing Bonus 2 Below:-

In [ ]:
import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # use 'cuda' for any GPU

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# %pip install --quiet transformers==4.36.2 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 auto-gptq==0.4.2
# ^-- if this fails, remove the specific version and install latest ones: pip install --update transformers accelerate ...
!wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/2023/week_extra/retrieval/data.py -O data.py
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad-v2.0.json 2> log

import data
# backup download link: https://www.dropbox.com/s/q4fuihaerqr0itj/squad.tar.gz?dl=1
train, test = data.build_dataset('./squad-v2.0.json')

--2025-05-07 15:41:48--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/2023/week_extra/retrieval/data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3553 (3.5K) [text/plain]
Saving to: ‘data.py’

data.py             100%[===================>]   3.47K  --.-KB/s    in 0s      

2025-05-07 15:41:48 (48.5 MB/s) - ‘data.py’ saved [3553/3553]



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline

model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert = AutoModel.from_pretrained(model_name)

2025-05-07 15:42:06.759864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746632527.194822      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746632527.322772      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
class Vectorizer(nn.Module):
    def __init__(self, hid_size=256, bert=bert):
        """ A small feedforward network on top of pre-trained encoder. 2-3 layers should be enough """
        super().__init__()
        self.bert = bert
        self.hid_size = hid_size

        # define a few layers to be applied on top of pre-trained BERT
        # note: please make sure your final layer comes with _linear_ activation
        # <YOUR CODE HERE>
        # Freeze BERT parameters
        for param in self.bert.parameters():
            param.requires_grad = False

        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, hid_size)
        self.relu = nn.ReLU()
        self.to(device)


    def forward(self, input_phrases):
        """
        Apply vectorizer. Use dropout and any other hacks at will.
        :param input_phrases: a list of strings, [batch_size]
        :returns: predicted phrase vectors, [batch_size, output_size]

        Note: you may want to use dropouts.
        if self.training:
          <something>

        Note 2: you may also want to use with torch.no_grad to avoid training BERT for your first attempts

        """

        # In fact, please DO use at least 10% dropout!
        # <YOUR CODE>
        # return <...>
        with torch.no_grad():
            batch = tokenizer(input_phrases, padding=True, truncation=True, return_tensors="pt").to(device)
            outputs = self.bert(**batch)

        mask = batch['attention_mask'].unsqueeze(-1).float()
        token_embs = outputs.last_hidden_state
        phrase_embs = (token_embs * mask).sum(1) / mask.sum(1)

        x = self.dropout(phrase_embs)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [ ]:
def similarity(a, b):
    """ Dot product as a similarity function """
    # <YOUR CODE>
    # return <...>
    return torch.sum(a * b, dim=1)


def compute_loss(question_vectors, correct_answer_vectors, wrong_answer_vectors, delta=1.0):
    """
    Compute the triplet loss as per formula above.
    Use similarity function above for  sim[a, b]
    :param question_vectors: float32[batch_size, vector_size]
    :param correct_answer_vectors: float32[batch_size, vector_size]
    :param wrong_answer_vectors: float32[batch_size, vector_size]
    :returns: loss for every row in batch, float32[batch_size]
    Hint: you can compute max(0, *) using torch.relu :)
    """
    # <YOUR CODE>
    # return <...>
    pos_sim = similarity(question_vectors, correct_answer_vectors)
    neg_sim = similarity(question_vectors, wrong_answer_vectors)
    print(torch.relu(delta - pos_sim + neg_sim))
    return torch.relu(delta - pos_sim + neg_sim)

In [ ]:
from huggingface_hub import login
login(token="login_taken_hidden")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import json

# Initialize semantic similarity scorer
scorer = SentenceTransformer('all-MiniLM-L6-v2')

# Model configuration with architecture specification
models = {
    # Instruct Models
    # "Mistral-7B-Instruct": ("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ", "causal", "instruct"),
    "FLAN-T5-Base": ("google/flan-t5-base", "seq2seq", "instruct"),
    # Non-Instruct Models
    # "Mistralai/Mistral-7B-v0.1": ("mistralai/Mistral-7B-v0.1", "causal", "non-instruct"),
    # "Tiiuae/Falcon-1b":("tiiuae/falcon-1b", "causal", "non-instruct"),
    "DistilGPT-2": ("distilgpt2", "causal", "non-instruct"),
    # "Mosaicml/Mpt-7b":("mosaicml/mpt-7b", "causal", "non-instruct"),
    "Phi-2": ("microsoft/phi-2", "causal", "non-instruct")
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = []

import random

def iterate_minibatches(data, batch_size, shuffle=True, cycle=False):
    """
    Generates minibatches of triples: {questions, correct answers, wrong answers}
    If there are several wrong (or correct) answers, picks one at random.
    """
    indices = np.arange(len(data))
    while True:
        if shuffle:
            indices = np.random.permutation(indices)
        for batch_start in range(0, len(indices), batch_size):
            batch_indices = indices[batch_start: batch_start + batch_size]
            batch = data.iloc[batch_indices]
            questions = batch['question'].values
            correct_answers = np.array([
                row['options'][random.choice(row['correct_indices'])]
                for i, row in batch.iterrows()
            ])
            wrong_answers = np.array([
                row['options'][random.choice(row['wrong_indices'])]
                for i, row in batch.iterrows()
            ])

            yield {
                'questions' : questions,
                'correct_answers': correct_answers,
                'wrong_answers': wrong_answers,
            }
        if not cycle:
            break

def format_prompt(model_arch, model_type, context, question):
    """Format prompt based on model architecture and type"""
    if model_type == "instruct":
            messages = [
                {"role": "user", "content": "Question: Who wrote The Racial Contract?\nContext: This led to a challenge to the social contract by philosophers Charles W. Mills in his book The Racial Contract and Carole Patemen in her book The Sexual Contract that the social contract excluded persons of colour and women respectively."},
                {"role": "assistant", "content": "Charles W. Mills wrote The Racial Contract."},

                {"role": "user", "content": "Question: Where was the centre of Argentoratum situated?\nContext: The centre of Argentoratum proper was situated on the Grande Île (Cardo: current Rue du Dôme, Decumanus: current Rue des Hallebardes)."},
                {"role": "assistant", "content": "The Grande Île (current Rue du Dôme and Rue des Hallebardes) was the center of Argentoratum."},

                {"role": "user", "content": "Question: Ice from what source helped to create Earth's oceans?\nContext: Condensing water vapor, augmented by ice delivered from comets, produced the oceans."},
                {"role": "assistant", "content": "Comets contributed ice to create Earth's oceans."},

                {"role": "user", "content": "Question: What occurs with osmotic diarrhea?\nContext: Osmotic diarrhea occurs when too much water is drawn into the bowels."},
                {"role": "assistant", "content": "Excess water drawn into the bowels occurs in osmotic diarrhea."},

                {"role": "user", "content": "Question: What is the Premier League commonly referred to outside the UK?\nContext: Outside the UK it is commonly referred to as the English Premier League (EPL)."},
                {"role": "assistant", "content": "The English Premier League (EPL) is its common name outside the UK."},

                {"role": "user", "content": "Question: Where did industrial music get its name?\nContext: Throbbing Gristle's independent label Industrial Records would become a hub for this scene and provide it with its namesake."},
                {"role": "assistant", "content": "Industrial Records (Throbbing Gristle's label) gave industrial music its name."},

                {"role": "user", "content": "Question: Who recorded \"You Really Got Me\"?\nContext: Early forms of hard rock can be heard in the work of Chicago blues musicians Elmore James, Muddy Waters, and Howlin' Wolf, The Kingsmen's version of \"Louie Louie\" (1963) which made it a garage rock standard, and the songs of rhythm and blues influenced British Invasion acts, including \"You Really Got Me\" by The Kinks (1964)."},
                {"role": "assistant", "content": "The Kinks recorded \"You Really Got Me\"."},

                {"role": "user", "content": "Question: What executive initially led the Arctic drilling project?\nContext: At inception, the project was led by Pete Slaiby, a Shell executive who had previously worked in the North Sea."},
                {"role": "assistant", "content": "Pete Slaiby (Shell executive) initially led the Arctic drilling project."},

                {"role": "user", "content": "Question: In a direct review, what type of court looks at the record?\nContext: An appellate court examines the record of evidence presented in the trial court and the law that the lower court applied and decides whether the decision was legally sound or not."},
                {"role": "assistant", "content": "An appellate court reviews records in direct review cases."},

                {"role": "user", "content": "Question: Whose responsibility is it to conduct meetings and decide action for cases involving sexual misconduct?\nContext: Elders maintain general responsibility for congregational governance, setting meeting times, selecting speakers and conducting meetings, directing the public preaching work, and creating \"judicial committees\" to investigate and decide disciplinary action for cases involving sexual misconduct or doctrinal breaches."},
                {"role": "assistant", "content": "Elders conduct meetings for sexual misconduct cases."},

                {"role": "user", "content": "Question: What do Ethnoarchaeologists gain a better understanding of by studying living human groups?\nContext: Ethnoarchaeology is a type of archaeology that studies the practices and material remains of living human groups in order to gain a better understanding of the evidence left behind by past human groups, who are presumed to have lived in similar ways."},
                {"role": "assistant", "content": "Past human practices are better understood via ethnoarchaeology."},

                {"role": "user", "content": "Question: In what form is the information or content sent in a simple model?\nContext: A message in natural language is sent in some form (as spoken language) from an emisor/sender/encoder to a destination/receiver/decoder."},
                {"role": "assistant", "content": "Natural language (e.g., spoken words) is used in simple models."},

                {"role": "user", "content": "Question: What three departments of the inner suburbs are included in the metropole?\nContext: It includes the City of Paris, plus the communes of the three departments of the inner suburbs: Hauts-de-Seine, Seine-Saint-Denis and Val-de-Marne."},
                {"role": "assistant", "content": "Hauts-de-Seine, Seine-Saint-Denis, and Val-de-Marne are included."},

                {"role": "user", "content": "Question: From where did the ideology of the Liberals stem?\nContext: More broadly, the Liberal Party's ideological ancestry stretched back to the anti-Labor groupings in the first Commonwealth parliaments."},
                {"role": "assistant", "content": "Anti-Labor Commonwealth groupings inspired Liberal ideology."},

                {"role": "user", "content": "Question: What did the Tuvalu Prime Minister say was not enough of an immediate threat to cause evacuation of the population?\nContext: In 2006 Maatia Toafa (Prime Minister from 2004–2006) said his government did not regard rising sea levels as such a threat that the entire population would need to be evacuated."},
                {"role": "assistant", "content": "Rising sea levels were deemed insufficiently urgent by Prime Minister Maatia Toafa."},

                {"role": "user", "content": "Question: In 1995, who decided to manage the girls singing group?\nContext: In 1995 Beyoncé's father resigned from his job to manage the group."},
                {"role": "assistant", "content": "Beyoncé's father managed the girls' group in 1995."},

                {"role": "user", "content": "Question: Which parts of annelids are different from true segments?\nContext: The frontmost and rearmost sections are not regarded as true segments as they do not contain the standard sets of organs and do not develop in the same way as the true segments."},
                {"role": "assistant", "content": "Frontmost/rearmost sections differ from true annelid segments."},

                {"role": "user", "content": "Question: Why was secondary school introduced in the U.S?\nContext: Secondary education in the United States did not emerge until 1910, with the rise of large corporations and advancing technology in factories, which required skilled workers."},
                {"role": "assistant", "content": "Skilled labor demands (1910 factories/corporations) prompted U.S. secondary schools."},

                {"role": "user", "content": "Question: When did the Arabs invade Iran?\nContext: The prolonged Byzantine-Sassanid Wars opened the way for an Arab invasion to Iran in the 7th century."},
                {"role": "assistant", "content": "7th century Arab invasion occurred during Byzantine-Sassanid conflicts."},

                {"role": "user", "content": "Question: What has been more prominently reported in recent years in Saint Helena?\nContext: There have been reports of child abuse in St Helena."},
                {"role": "assistant", "content": "Child abuse reports have risen in Saint Helena recently."},
                {"role": "user", "content": f"Question: {question}\nContext:\n{context}"}
                        ]
            return messages



    else:
        qa_text = f""" Answer similar to the following examples
        Question: Who wrote The Racial Contract? Context: This led to a challenge to the social contract by philosophers Charles W. Mills in his book The Racial Contract.
        Answer: Charles W. Mills wrote The Racial Contract.

        Question: Where was the centre of Argentoratum situated? Context: The centre was on the Grande Île (current Rue du Dôme, Rue des Hallebardes).
        Answer: The Grande Île (current Rue du Dôme and Rue des Hallebardes) was the center of Argentoratum.

        Question: Ice from what source helped to create Earth's oceans? Context: Condensing vapor augmented by comet ice.
        Answer: Comets contributed ice to create Earth's oceans.

        Question: What occurs with osmotic diarrhea? Context: Too much water drawn into bowels.
        Answer: Excess water drawn into the bowels occurs.

        Question: Premier League's name outside UK? Context: Commonly called English Premier League (EPL).
        Answer: The English Premier League (EPL) is its common name.

        Question: Industrial music's name origin? Context: Throbbing Gristle's Industrial Records label.
        Answer: Industrial Records (Throbbing Gristle's label) gave the name.

        Question: Who recorded "You Really Got Me"? Context: The Kinks' 1964 version.
        Answer: The Kinks recorded "You Really Got Me".

        Question: {question} Context: {context}
        Answer:
        """
        #Couldn't add more examples as it is giving error.
        return qa_text

# Main evaluation loop
for model_name, (model_id, arch, model_type) in models.items():
    print(f"\nEvaluating {model_name}...")

    try:
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        # Load correct model class based on architecture
        if arch == "causal":
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                # device_map='auto',
                torch_dtype=torch.float16,
                low_cpu_mem_usage=True,
                offload_state_dict=True).to(device)
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
        elif arch == "seq2seq":
            model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

        model.eval()

        semantic_scores = []

        # Get test minibatch
        minibatch = next(iterate_minibatches(train, batch_size=20))
        questions = minibatch['questions']
        answers = minibatch['correct_answers']

        for question, context in zip(questions, answers):
            # Format prompt
            prompt = format_prompt(arch, model_type, context, question)
            if arch=="seq2seq":
                # Tokenize and generate
                messages_string = json.dumps(prompt, indent=4)
                # input_ids = tokenizer.apply_chat_template(
                #     prompt,
                #     return_tensors="pt"
                # ).to(device)
                # with torch.no_grad():
                #     outputs = model.generate(
                #         input_ids,
                #         max_new_tokens=50,
                #         pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
                #     )
                inputs = tokenizer(messages_string, return_tensors="pt").to(device)
                with torch.no_grad():
                        outputs = model.generate(
                            inputs.input_ids,
                            max_new_tokens=50,
                            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
                        )
            # Generate response
            else:
                inputs = tokenizer(prompt, return_tensors="pt").to(device)
                with torch.no_grad():
                    outputs = model.generate(
                        inputs.input_ids,
                        max_new_tokens=50,
                        pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
                    )

            # Decode and score
            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"Model: {model_name}\nQuestion: {question}\nContext: {context}\nAnswer: {answer} ")
            sem_encodings = scorer.encode([answer, context])
            sem_score = F.cosine_similarity(
                torch.tensor(sem_encodings[0]),
                torch.tensor(sem_encodings[1]),
                dim=0
            ).item()
            semantic_scores.append(sem_score)

        # Save results
        results.append({
            "Model": model_name,
            "Architecture": arch.upper(),
            "Type": model_type.upper(),
            "Semantic Score": np.mean(semantic_scores),
        })

    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")
        continue



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Model: Phi-2
Question: What is the doctrine that says that the mind and only the mind are real?
Context: While the Mādhyamaka school held that asserting the existence or non-existence of any ultimately real thing was inappropriate, some exponents of Yogacara asserted that the mind and only the mind is ultimately real (a doctrine known as cittamatra).
Answer:  Answer similar to the following examples
        Question: Who wrote The Racial Contract? Context: This led to a challenge to the social contract by philosophers Charles W. Mills in his book The Racial Contract.
        Answer: Charles W. Mills wrote The Racial Contract.
        
        Question: Where was the centre of Argentoratum situated? Context: The centre was on the Grande Île (current Rue du Dôme, Rue des Hallebardes).
        Answer: The Grande Île (current Rue du Dôme and Rue des Hallebardes) was the center of Argentoratum.
        
        Question: Ice from what source helped to create Earth's oceans? Context: Condens

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Model: Phi-2
Question: When did Favre make the declaration concerning colonial territories?
Context: While the republican government was amenable to war reparations or ceding colonial territories in Africa or in South East Asia to Prussia, Favre on behalf of the Government of National Defense, declared on 6 September that France would not "yield an inch of its territory nor a stone of its fortresses."
Answer:  Answer similar to the following examples
        Question: Who wrote The Racial Contract? Context: This led to a challenge to the social contract by philosophers Charles W. Mills in his book The Racial Contract.
        Answer: Charles W. Mills wrote The Racial Contract.
        
        Question: Where was the centre of Argentoratum situated? Context: The centre was on the Grande Île (current Rue du Dôme, Rue des Hallebardes).
        Answer: The Grande Île (current Rue du Dôme and Rue des Hallebardes) was the center of Argentoratum.
        
        Question: Ice from what sour

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Display results
if results:
    df = pd.DataFrame(results)
    print("\nEvaluation Results:")
    print(df[['Model', 'Architecture', 'Type', 'Semantic Score']].to_markdown(index=False))
else:
    print("No models were successfully evaluated.")


Evaluation Results:
| Model        | Architecture   | Type         |   Semantic Score |
|:-------------|:---------------|:-------------|-----------------:|
| FLAN-T5-Base | SEQ2SEQ        | INSTRUCT     |        0.488178  |
| DistilGPT-2  | CAUSAL         | NON-INSTRUCT |        0.056546  |
| Phi-2        | CAUSAL         | NON-INSTRUCT |        0.0566531 |


# For some reason, the non-instruct models are not generating properly. Maybe because I used lower end ones. I tried better models, but insuffient memory to load more than one in the system to compare.  